In [ ]:
import pandas as pd

Objetivo: Leer archivo PKL olympic_results.pkl y compararlo con archivo CSV con el mismo nombre

In [ ]:
# CSV olympic_results
df_csv = pd.read_csv('datos/raw/olympic_results.csv')
df_csv

In [ ]:
# Pickle olympic_results
df_pkl = pd.read_pickle('datos/raw/olympic_results.pkl')
df_pkl

In [ ]:
def analizar_df(df):
    # Crear una lista para almacenar el resumen de las columnas
    resumen_data = []

    for col in df.columns:
        nombre = col
        tipo = df[col].dtype
        cantidad_valores = len(df[col])
        valores_unicos = df[col].apply(lambda x: str(x) if isinstance(x, list) else x).nunique()
        nulos = df[col].isnull().sum()
        strings_vacios = df[col].apply(lambda x: x == '').sum() if df[col].dtype == 'object' else 0

        # Tratar de obtener el valor más frecuente
        try:
            mas_frecuente = df[col].mode().iloc[0] if not df[col].mode().empty else None
        except TypeError:
            mas_frecuente = None

        # Tratar de obtener el valor menos frecuente
        try:
            menos_frecuente = df[col].value_counts().idxmin() if not df[col].value_counts().empty else None
        except TypeError:
            menos_frecuente = None

        # Añadir la información de la columna al resumen
        resumen_data.append({
            'Nombre': nombre,
            'Tipo': tipo,
            'Cantidad de Valores': cantidad_valores,
            'Valores Únicos': valores_unicos,
            'Nulos': nulos,
            'Strings Vacíos': strings_vacios,
            'Más Frecuente': mas_frecuente,
            'Menos Frecuente': menos_frecuente
        })

    # Convertir la lista de resumen a un DataFrame
    resumen = pd.DataFrame(resumen_data)

    # Devolver el resumen
    return resumen


In [ ]:
analizar_df(df_csv)

In [ ]:
analizar_df(df_pkl)

Observamos diferencias en la cantidad de nulos y cadenas vacías.
Decidimos reemplazar los valores nulos por cadenas vacías para mantener un criterio en común entre los dos data frames

In [ ]:
# Reemplazar valores nulos por strings vacíos en todas las columnas
df_csv = df_csv.fillna('')

In [ ]:
# Reemplazar valores nulos por strings vacíos en todas las columnas
df_pkl = df_pkl.fillna('')

In [ ]:
analizar_csv_resultado = analizar_df(df_csv)
analizar_csv_resultado

In [ ]:
analizar_pkl_resultado = analizar_df(df_pkl)
analizar_pkl_resultado

Evaluamos las diferencias que continuan entre los dos data frames

In [ ]:
# Reset index y alinear - los DataFrames deben estar alineados correctamente antes de compararlos
df_csv_reset = df_csv.reset_index(drop=True)
df_pkl_reset = df_pkl.reset_index(drop=True)

df_csv_aligned, df_pkl_aligned = df_csv_reset.align(df_pkl_reset)

In [ ]:
differencias_analisis = analizar_csv_resultado.compare(analizar_pkl_resultado)
# Identificar las filas completas con diferencias
differences_indices = differencias_analisis.index
df_csv_diff = analizar_csv_resultado.loc[differences_indices]
df_pkl_diff = analizar_pkl_resultado.loc[differences_indices]

In [ ]:
# Mostrar las filas completas con diferencias
print("En el DataFrame CSV:")
df_csv_diff

In [ ]:
print("En el DataFrame PKL:")
df_pkl_diff

In [ ]:
def comparar_columnas(df1, df2):
    """
    Compara las columnas de dos DataFrames y muestra las diferencias.

    Args:
        df1 (pd.DataFrame): Primer DataFrame a comparar.
        df2 (pd.DataFrame): Segundo DataFrame a comparar.
    """
    # Verificar si ambos DataFrames tienen las mismas columnas
    if not df1.columns.equals(df2.columns):
        print("Los DataFrames no tienen las mismas columnas.")
        return

    for column in df1.columns:
        # Normalizar listas y tuplas en las columnas que contienen listas o tuplas
        if df1[column].apply(lambda x: isinstance(x, (list, tuple))).any():
            col1 = df1[column].apply(lambda x: sorted(x) if isinstance(x, (list, tuple)) else x)
            col2 = df2[column].apply(lambda x: sorted(x) if isinstance(x, (list, tuple)) else x)
            col1 = col1.apply(lambda x: str(x) if isinstance(x, (list, tuple)) else x)
            col2 = col2.apply(lambda x: str(x) if isinstance(x, (list, tuple)) else x)
        else:
            col1 = df1[column]
            col2 = df2[column]

        if col1.equals(col2):
            print(f"La columna '{column}' es idéntica en ambos DataFrames.")
        else:
            print(f"Diferencias encontradas en la columna '{column}':")
            differences = col1.compare(col2)
            print(differences)

In [ ]:
comparar_columnas(df_csv_aligned, df_pkl_aligned)

Se observan diferencias en las columnas 'athletes' y 'country_code'.

In [ ]:
#Evaluamos country_code
csv_country_code = df_csv_aligned['country_code']
pkl_country_code = df_pkl_aligned['country_code']

print(f"Diferencias encontradas en la columna 'country_code':")
diferencia_cc = csv_country_code.compare(pkl_country_code)
print(f"{len(diferencia_cc)} filas")
diferencia_cc

In [ ]:
# Identificar las filas completas con diferencias de country_code
diferencia_cc_indices = diferencia_cc.index
df_csv_diff_cc = df_csv_aligned.loc[diferencia_cc_indices]
df_pkl_diff_cc = df_pkl_aligned.loc[diferencia_cc_indices]

In [ ]:
# Mostrar las filas completas con diferencias
print("En el DataFrame CSV:")
df_csv_diff_cc

In [ ]:
print("En el DataFrame PKL:")
df_pkl_diff_cc

In [ ]:
comparar_columnas(df_csv_diff_cc, df_pkl_diff_cc)

In [ ]:
df_csv_diff_cc[['country_name', 'country_code', 'country_3_letter_code']]

In [ ]:
df_pkl_diff_cc[['country_name', 'country_code', 'country_3_letter_code']]

Observamos diferencias para el código del país "Namibia", probablemente causadas por la coincidencia con "Not Available" de Python. Decidimos aplicar el código de país "NA" a todas las filas con país "Namibia" en el df proveniente del CSV

In [ ]:
df_csv_aligned.loc[df_csv_aligned['country_name'] == 'Namibia', 'country_code'] = 'NA'

In [ ]:
comparar_columnas(df_csv_aligned, df_pkl_aligned)

In [ ]:
#Evaluamos athletes
csv_athletes = df_csv_aligned['athletes']
pkl_athletes = df_pkl_aligned['athletes']

print(f"Diferencias encontradas en la columna 'athletes':")
diferencia_at = csv_athletes.compare(pkl_athletes)
print(f"{len(diferencia_at)} filas")
diferencia_at

In [ ]:
# Identificar las filas completas con diferencias de athletes
diferencia_at_indices = diferencia_at.index
df_csv_diff_at = df_csv_aligned.loc[diferencia_at_indices]
df_pkl_diff_at = df_pkl_aligned.loc[diferencia_at_indices]

In [ ]:
# Mostrar las filas completas con diferencias
print("En el DataFrame CSV:")
df_csv_diff_at

In [ ]:
print("En el DataFrame PKL:")
df_pkl_diff_at

Se observa diferencia de formato la columna athletes

In [ ]:
def identificar_tipos(df, column_name):
    """
    Identifica y resume los tipos de datos de los elementos en una columna.

    Args:
        df (pd.DataFrame): DataFrame que contiene la columna.
        column_name (str): Nombre de la columna a inspeccionar.
    """
    types_summary = df[column_name].apply(lambda x: type(x).__name__).value_counts()
    print(f"Resumen de tipos de datos en la columna '{column_name}':")
    print(types_summary)

In [ ]:
identificar_tipos(df_csv_aligned, 'athletes')

In [ ]:
identificar_tipos(df_pkl_aligned, 'athletes')

In [ ]:
import ast

def convertir_string_a_list(x):
    """
    Convierte strings que representan listas o tuplas a objetos reales.
    """
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return x

In [ ]:
# Aplicar la función a la columna athletes proveniente del csv
df_csv_aligned['athletes_converted'] = df_csv_aligned['athletes'].apply(convertir_string_a_list)

In [ ]:
identificar_tipos(df_csv_aligned, 'athletes_converted')

In [ ]:
df_csv_aligned['athletes'] = df_csv_aligned['athletes_converted']

In [ ]:
df_csv_aligned = df_csv_aligned.drop(columns=['athletes_converted'])

In [ ]:
comparar_columnas(df_csv_aligned, df_pkl_aligned)

Confirmamos que los datos de estos archivos coinciden, y optamos por usar la versión PKL para evitar el error con el país "Namibia".

In [ ]:
df_pkl.to_csv('datos/raw/olympic_results_from_pkl.csv')